In [1]:
# Import Libraries
import math
import numpy as np
from backtesting import Backtest, Strategy
import pandas as pd

# Load Data
file_path = '../Datasets/SPX.csv'
data = pd.read_csv(file_path, parse_dates=['Date'])
data.set_index('Date', inplace=True)

# Ensure data is properly scaled for micro-shares
data['Close'] = data['Close'] * 10**-6

# Define the DCA Strategy class
class DCA(Strategy):
    amount_to_invest = 10

    def init(self):
        self.day_of_week = self.I(
            lambda x: x,
            self.data.Close.s.index.dayofweek,
            plot=False
        )

    def next(self):
        if self.day_of_week[-1] == 1:  # Buy on Tuesday
            size = self.amount_to_invest / self.data.Close[-1]
            if size > 0:  # Ensure size is positive
                self.buy(size=math.floor(size))
            try:
                if self.data.Close[-1] / self.data.Close[-30] < 0.95:
                    size = self.amount_to_invest / self.data.Close[-1]
                    if size > 0:  # Ensure size is positive
                        self.buy(size=math.floor(size))
            except IndexError:
                pass

# Run the backtest
bt = Backtest(
    data,
    DCA,
    trade_on_close=True,
    cash=10000  # ensure enough cash for all buys
)

stats = bt.run(amount_to_invest=10)  # set the investment amount
bt.plot()

# Calculate and display metrics
trades = stats['_trades']
price_paid = trades['Size'] * trades['EntryPrice']
total_invested = price_paid.sum()

current_shares = trades['Size'].sum()
current_equity = current_shares * data.Close.iloc[-1]

print("Total investment:", total_invested)
print("Current Shares:", current_shares / (10**6))
print("Current Equity:", current_equity)
print("Return:", current_equity / total_invested)


C:\Users\23713\anaconda3\envs\pytorch\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

C:\Users\23713\anaconda3\envs\pytorch\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\23713\anaconda3\envs\pytorch\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Total investment: 5639.201806778452
Current Shares: 2.047646
Current Equity: 9766.923479896388
Return: 1.7319691358015097


In [2]:
stats

Start                     2014-01-02 00:00:00
End                       2023-12-29 00:00:00
Duration                   3648 days 00:00:00
Exposure Time [%]                   99.841017
Equity Final [$]                 14155.405888
Equity Peak [$]                  14155.405888
Return [%]                          41.554059
Buy & Hold Return [%]              160.364749
Return (Ann.) [%]                    3.541923
Volatility (Ann.) [%]                 7.75269
Sharpe Ratio                         0.456864
Sortino Ratio                        0.658774
Calmar Ratio                         0.223877
Max. Drawdown [%]                  -15.820855
Avg. Drawdown [%]                   -0.574128
Max. Drawdown Duration      709 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                  564
Win Rate [%]                         99.64539
Best Trade [%]                     172.524512
Worst Trade [%]                     -0.212577
Avg. Trade [%]                    